##Introduction

This notebook contains the second case study of Google's Data Analytics Professional Certificate. The case study requires the analyst to follow to the steps of the data analysis process (ask, prepare, process, analyze, share,and act) using FitBit Fitness Tracker data to formulate high-level marketing recomendations for Bellabeat, a high-tech manufacturer of health-focused products for women. 

##Ask

Bellabeat's business task is to unlock new growth opportunities by analyzing smart device fitness data and gaining insight into how consumers use their smart devices, which will help guide the marketing strategy for the company. To conduct this analysis, we will be examining what are some trends in smart device usage, how these trends couple apply to Bellabeat customers, and how could these trends help influence Bellabeat marketing strategy? For deliverables, we will be making three high-level recomendations for the marketing team.

##Prepare

These datasets were generated by respondents to a distributed survey via Amazon Mechanical Turk between 03/12/2016 to 05/12/2016. Thirty eligible Fitbit users consented to the submission of personal tracker data, including second-level, minute-level, hour-level, and day-level output for heart rate, burned calories, activity intensity, MET, steps, and sleep activity. Each data point is associated with a timestamp and a user by their id.

In terms of limitations, the data consists of 30 users within a 30 day period from 2016. This makes the source outdated and the sample size too small for proper analysis. There is no demographic information to guarantee the data is representative of female users as Bellabeat's target audience.

In [45]:
#install packages
library(tidyverse)
library(lubridate)

In [46]:
#import datasets that we will use in the analysis
daily_activity <- read.csv('../input/fitbit/Fitabase Data 4.12.16-5.12.16/dailyActivity_merged.csv')
hourly_steps <- read.csv('../input/fitbit/Fitabase Data 4.12.16-5.12.16/hourlySteps_merged.csv')
daily_sleep <- read.csv('../input/fitbit/Fitabase Data 4.12.16-5.12.16/sleepDay_merged.csv')

##Process

I utilized both SQL and R to clean and organize my data. We'll first look at my queries in SQL.

In SQL, I first converted the schema of the daily sleep data's date column into a useable format (from timestamp to datetime).

In [47]:
WITH daily_activity AS (
  SELECT CURRENT_TIMESTAMP() ActivityDate
)
SELECT *, CAST(ActivityDate AS DATETIME) ActivityDate
FROM `geometric-vim-314715.Bellabox_Marketing_Analysis.daily_activity` AS daily_activity


ERROR: Error in parse(text = x, srcfile = src): <text>:1:6: unexpected symbol
1: WITH daily_activity
         ^


I then utilized a JOIN operation to combine sleep data and daily activity data.

In [ ]:
SELECT  
daily_sleep.SleepDay_1,
daily_sleep.TotalMinutesAsleep,
daily_activity.TotalDistance,
daily_activity.VeryActiveMinutes,
daily_activity.FairlyActiveMinutes,
daily_activity.LightlyActiveMinutes,
daily_activity.SedentaryMinutes,
daily_activity.Calories,
daily_activity.total_minutes
FROM `geometric-vim-314715.Bellabox_Marketing_Analysis.daily_sleep_new` AS daily_sleep
LEFT JOIN `geometric-vim-314715.Bellabox_Marketing_Analysis.daily_activity_new` AS daily_activity
ON daily_sleep.Id=daily_activity.Id and daily_sleep.SleepDay_1=daily_activity.ActivityDate_1

I then saved and imported the dataset from SQL to RStudio

In [ ]:
#import dataset from SQL Join operation
sleep_by_activity <- read.csv('../input/fitbit-fitness-tracker-daily-activity-and-sleep/sleep_by_activity.csv')

In R, I first checked my data frames to ensure clean data.

In [ ]:
str(hourly_steps)
str(daily_activity)
str(daily_sleep)
str(sleep_by_activity)

I then converted the date column's data type from character to date time in each of the data frames.

In [ ]:
hourly_steps$ActivityHour <- parse_date_time(hourly_steps$ActivityHour, '%m/%d/%Y %I:%M:%S %p')
daily_activity$ActivityDate <- parse_date_time(daily_activity$ActivityDate, '%m/%d/%Y')
daily_sleep$SleepDay <- parse_date_time(daily_sleep$SleepDay, '%m/%d/%Y %I:%M:%S %p')
sleep_by_activity$SleepDay <- parse_date_time(sleep_by_activity$SleepDay, '%m/%d/%Y')

To analyze the data by time of day (i.e. 1:00 AM, 2:00 AM, etc.), I then created a new column for time of day for the hourly steps data frame.

In [ ]:
hourly_steps$Time <- format(as.POSIXlt(hourly_steps$ActivityHour), "%H:%M:%S")

To analyze the data by day of the week (i.e. Monday, Tuesday, etc.), I added a column for the day of the week for each observation.

In [ ]:
hourly_steps$day_of_week <- format(as.Date(hourly_steps$ActivityHour), "%A")
hourly_steps$day_of_week <- ordered(hourly_steps$day_of_week, levels=c("Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"))


daily_activity$day_of_week <- format(as.Date(daily_activity$ActivityDate), "%A")
daily_activity$day_of_week <- ordered(daily_activity$day_of_week, levels=c("Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"))

sleep_by_activity$day_of_week <- format(as.Date(sleep_by_activity$SleepDay), "%A")
sleep_by_activity$day_of_week <- ordered(sleep_by_activity$day_of_week, levels=c("Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"))

Lastly, I created a column for the total minutes recorded per day for the daily activity data frame.

In [ ]:
daily_activity$total_minutes <- rowSums(daily_activity [,11:14])

##Analyze

I started my analysis by aggregating some data to find some averages. 

I first focused on how much time people dedicated to different levels of activity each day, from sedentary to very active. 

In [ ]:
activity_distribution_per_day <- aggregate(cbind(daily_activity$total_minutes, daily_activity$SedentaryMinutes, daily_activity$LightlyActiveMinutes, daily_activity$FairlyActiveMinutes, daily_activity$VeryActiveMinutes) ~ daily_activity$day_of_week, FUN = mean)
write_csv(activity_distribution_per_day, "~/Desktop/activity_distribution_per_day.csv")

Lastly, I focused on the average number of steps per hour of the day.

In [ ]:
steps_per_hour <- aggregate(hourly_steps$StepTotal ~ hourly_steps$Time, FUN = mean)
write_csv(steps_per_hour, "~/Desktop/step_per_hour.csv")

##Share

I then utlized the ggplot2 package to build some data visualizations for analysis. This will help stakeholders get a better idea of the analysis before I present my recommendations.

The first plot revelaed the relationship between daily average "active" minutes (lightly + fairly + very active minutes) and daily average calories burned.

In [ ]:
ggplot(data = daily_activity, aes(x= VeryActiveMinutes + FairlyActiveMinutes + LightlyActiveMinutes, y = Calories)) + geom_point() + geom_smooth(method  = "lm") + labs(title = "Daily Non-Sedentary Activity and Calories Burned") + xlab("Daily Non-Sedentary Minutes (Fairly + Fairly + Very Active") + ylab("Daily Calories Burned")

I compared this plot to another plot that looked at the relationship between daily average sedentary minutes and daily average calories burned.

In [ ]:
ggplot(data = daily_activity, aes(x = SedentaryMinutes, y = Calories)) + geom_point() + geom_smooth(method = "lm") + labs(title = "Daily Sedentary Minutes and Calories Burned") + xlab("Daily Sedentary Minutes") + ylab("Daily Calories Burned")

I then wanted to factor in how the daily average minutes asleep related to the daily average calories burned.

In [ ]:
ggplot(data = sleep_by_activity, aes(x = TotalMinutesAsleep, y = Calories)) + geom_point() + geom_smooth() + labs(title = "Daily Minutes Slept and Calories Burned") + xlab("Daily Minutes Slept") + ylab("Daily Calories Burned")

Then, I examined how the daily average of minutes slept realted to the daily average of fairly active and very active minutes.

In [ ]:
ggplot(data = sleep_by_activity, aes(x = TotalMinutesAsleep, y = VeryActiveMinutes + FairlyActiveMinutes)) + geom_point() + geom_smooth() + labs(title = "Daily Minutes Slept and Active Minutes") + xlab("Daily Minutes Slept") + ylab("Daily Active Minutes")

Lastly, I looked at the how the daily average of minutes slept related to the the the daily average of sedentary minutes.

In [ ]:
ggplot(data = sleep_by_activity, aes(x = TotalMinutesAsleep, y = SedentaryMinutes)) + geom_point() + geom_smooth() + labs(title = "Daily Minutes Slept and Sedentary Minutes") + xlab("Daily Minutes Slept") + ylab("Daily Sedentary Minutes")

After creating these plots in RStudio, I turned to Google Sheets. I first imported the aggregate data frames that I created earlier in RStudio. I then created pivot tables to organize the data and created charts to visualize the data.

##Act

For the last step of the data analysis process, I will present some key findings and then share three high-level recomendations to the marketing team.

###Key Findings

1. There is a clear pattern between the amount of sleep and staying active and thus burning calories, although not many smart device users are tracking their sleep.
2. The longer someone stay sedentary, the less calories they will burn throughout the day.
3. On average, fitness tracker users are spending 80% of their day in a sedentary state.
4. People are walking more around 4 to 7 PM everyday and less active from 12 to 5 AM.

###Recommendations

1. Encouraging users to track their sleep to encourage a health lifestyle
2. Advertise the connection between getting enough activity each day and maintaining proper health and wellness goals.
3. Focus advertising to people who are more active in the evening, likely people who work during the day.